In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame
import sklearn

In [4]:
milk_test = pd.read_csv("cattle_data_test.csv")
milk_test.head()

,Cattle_ID,Breed,Climate_Zone,Management_System,Age_Months,Weight_kg,Parity,Lactation_Stage,Days_in_Milk,Feed_Type,Feed_Quantity_kg,Feeding_Frequency,Water_Intake_L,Walking_Distance_km,Grazing_Duration_hrs,Rumination_Time_hrs,Resting_Hours,Ambient_Temperature_C,Humidity_percent,Housing_Score,FMD_Vaccine,Brucellosis_Vaccine,HS_Vaccine,BQ_Vaccine,Anthrax_Vaccine,IBR_Vaccine,BVD_Vaccine,Rabies_Vaccine,Previous_Week_Avg_Yield,Body_Condition_Score,Milking_Interval_hrs,Date,Farm_ID,Feed_Quantity_lb,Mastitis
0,1,Holstein,Temperate,Semi_Intensive,42,420.9,3,Early,302,Mixed_Feed,13.026220,2,76.874069,2.22,5.2,-1.803060,10.6,-11.064474,27.4,0.908479,1,1,1,1,1,1,0,0,6.76,3.5,12,2022-11-29,FARM_0740,28.4445,0
1,2,Holstein,Temperate,Extensive,112,466.6,5,Late,103,Green_Fodder,7.157062,5,70.059713,3.13,5.4,1.419651,8.9,21.713782,75.3,0.858894,1,0,1,0,0,0,0,1,9.87,2.0,12,2022-10-11,FARM_0607,15.2145,0
2,3,Jersey,Temperate,Semi_Intensive,57,496.5,1,Early,342,Crop_Residues,16.825069,4,96.759408,1.22,2.3,-4.383302,10.2,20.599418,68.0,0.826985,0,1,1,1,1,0,1,1,4.27,3.5,12,2024-03-21,FARM_0706,37.4850,0
3,4,Holstein,Subtropical,Extensive,63,617.5,6,Early,170,Concentrates,10.457681,5,97.842001,2.61,7.9,-8.808053,7.7,15.514841,39.0,0.416641,1,1,1,0,0,0,1,0,0.00,2.5,12,2023-12-25,FARM_0673,23.1525,0
4,5,Guernsey,Tropical,Semi_Intensive,34,413.9,6,Early,246,Dry_Fodder,16.176790,3,94.181886,2.48,6.6,-6.828832,5.5,23.140988,33.3,0.407842,1,1,1,0,1,0,1,1,1.91,3.5,12,2023-05-18,FARM_0112,35.5005,0


In [22]:
#categoricals
breed_data = milk_test['Breed']
climate_data = milk_test['Climate_Zone']
manage_data = milk_test['Management_System']
lactate_data = milk_test['Lactation_Stage']
feed_data = milk_test['Feed_Type']
print(climate_data.value_counts(dropna = False))
print()
print(breed_data.value_counts(dropna = False))
print()
print(manage_data.value_counts(dropna = False))

Climate_Zone
Arid             6817
Temperate        6762
Mediterranean    6632
Tropical         6602
Subtropical      6600
Continental      6587
Name: count, dtype: int64

Breed
Holstein        19926
Jersey           8091
Brown Swiss      5989
Guernsey         5963
Holstien           14
 Brown Swiss       10
Brown Swiss         7
Name: count, dtype: int64

Management_System
Intensive         8103
Mixed             8077
Semi_Intensive    7998
Pastoral          7919
Extensive         7903
Name: count, dtype: int64


In [33]:
print(lactate_data.value_counts(dropna = False))
print()
print(feed_data.value_counts(dropna = False))

Lactation_Stage
Mid      16115
Late     12027
Early    11858
Name: count, dtype: int64

Feed_Type
Concentrates     5052
Hay              5042
Green_Fodder     5021
Dry_Fodder       5012
Crop_Residues    4998
Pasture_Grass    4986
Silage           4964
Mixed_Feed       4925
Name: count, dtype: int64


In [42]:
milk_test['Breed'] = milk_test['Breed'].str.strip()
milk_test['Breed'] = milk_test['Breed'].replace({'Holstien': 'Holstein'})
print(milk_test['Breed'].value_counts())

Breed
Holstein       19940
Jersey          8091
Brown Swiss     6006
Guernsey        5963
Name: count, dtype: int64
